# CARLA Autonomous Vehicle with LiDAR-Camera Fusion

**AI-controlled vehicle using LiDAR-to-camera projection for obstacle detection and navigation**

Output: `output/YYYY-MM-DD_HH-MM-SS/lidar_projection/camera_{front,left,right}/`

## 1. Imports & Server Connection

In [1]:
import time
import carla
import numpy as np

from src import (
    Config, connect_to_carla, setup_synchronous_mode, restore_async_mode,
    clear_environment, reload_environment, spawn_vehicle, spawn_cameras,
    spawn_lidar, create_sensor_queues, project_lidar_to_camera,
    save_projection_image, update_spectator_view,
    lidar_density_to_waypoint,
)

from src.sensors import create_camera_callback, create_lidar_callback
from src.projection import image_to_array
from agents.navigation.controller import VehiclePIDController

config = Config()

# Connect to CARLA and enable synchronous mode
client, world = connect_to_carla(config)
original_settings, traffic_manager = setup_synchronous_mode(world, client, config)

print(f"✓ Connected to CARLA | Map: {world.get_map().name} | {config.fps} FPS sync mode")

✓ Connected to CARLA | Map: Carla/Maps/Town10HD_Opt | 20 FPS sync mode


---

## 2. Main Pipeline

Setup environment, spawn vehicle & sensors, run simulation with LiDAR-based steering.

### 1. Setup

In [2]:
# Environment setup (optional: clear/reload)
clear_environment(world)  # Remove buildings, props, etc.
# reload_environment(world)  # Uncomment to restore

# Spawn vehicle and sensors
vehicle = spawn_vehicle(world, config)
cameras, K = spawn_cameras(world, vehicle, config)
lidar = spawn_lidar(world, vehicle, config)
update_spectator_view(world, vehicle)
world.tick()

# Setup sensor queues and output directory
camera_queues, lidar_queue = create_sensor_queues(cameras)
output_dir = config.get_output_dir()
output_dirs = config.create_output_dirs(output_dir)

for cam_name, cam in cameras.items():
    cam.listen(create_camera_callback(camera_queues, cam_name))
lidar.listen(create_lidar_callback(lidar_queue))

# Initialize PID controller for smooth steering and speed control
pid_controller = VehiclePIDController(
    vehicle,
    args_lateral=config.args_lateral,
    args_longitudinal=config.args_longitudinal,
    max_throttle=config.max_throttle,
    max_brake=config.max_brake,
    max_steering=config.max_steering
)

print(f"✓ Vehicle + {len(cameras)} cameras + LiDAR spawned")
print(f"✓ PID Controller initialized (target: {config.target_speed_kmh} km/h)")
print(f"✓ Output: {output_dir}/")

✓ Vehicle + 3 cameras + LiDAR spawned
✓ PID Controller initialized (target: 20.0 km/h)
✓ Output: output/2026-02-02_22-45-27/


### 2. Simulation

In [3]:
# Free waypoint class - bypasses road snapping for pure LiDAR control
class FreeWaypoint:
    """Waypoint that doesn't snap to road - goes wherever LiDAR tells it"""
    def __init__(self, location, rotation):
        self.transform = carla.Transform(location, rotation)

# Run simulation with LiDAR-based PID steering
frame_counter, tick_count = 0, 0
current_projection_counts = {}

print(f"✓ Running {config.simulation_duration}s | Target speed: {config.target_speed_kmh} km/h | Controller: PID\n")

try:
    while tick_count < config.total_ticks:
        world.tick()
        tick_count += 1
        
        # Wait for sensor data
        start = time.time()
        while time.time() - start < 2.0:
            if all(camera_queues[c].qsize() > 0 for c in cameras) and lidar_queue.qsize() > 0:
                break
            time.sleep(0.001)
        
        should_save = (tick_count % config.capture_interval_ticks == 0)
        
        # Collect and clear queues
        camera_images = {n: camera_queues[n].get() for n in cameras if not camera_queues[n].empty()}
        for q in camera_queues.values():
            while not q.empty(): q.get()
        lidar_data = lidar_queue.get() if not lidar_queue.empty() else None
        while not lidar_queue.empty(): lidar_queue.get()
        
        # Process sensors
        if len(camera_images) == len(cameras) and lidar_data:
            projection_counts = {}
            for cam_name in cameras:
                proj_array, _, weighted_density = project_lidar_to_camera(
                    lidar_data, lidar, cameras[cam_name], image_to_array(camera_images[cam_name]), K, config)
                projection_counts[cam_name] = weighted_density
                if should_save:
                    save_projection_image(cam_name, proj_array, frame_counter, output_dirs)
            
            if should_save:
                frame_counter += 1
            current_projection_counts = projection_counts
        
        # PID-based LiDAR steering
        if current_projection_counts:
            # Step 1: Convert LiDAR density to target waypoint (local coordinates)
            waypoint_local, direction, analysis = lidar_density_to_waypoint(
                current_projection_counts, vehicle, config
            )
            
            # Step 2: Convert local waypoint to world coordinates
            vehicle_transform = vehicle.get_transform()
            forward_vec = vehicle_transform.get_forward_vector()
            right_vec = vehicle_transform.get_right_vector()
            vehicle_loc = vehicle_transform.location
            
            world_x = vehicle_loc.x + waypoint_local[0] * forward_vec.x + waypoint_local[1] * right_vec.x
            world_y = vehicle_loc.y + waypoint_local[0] * forward_vec.y + waypoint_local[1] * right_vec.y
            world_z = vehicle_loc.z
            
            # Step 3: Create FreeWaypoint directly from LiDAR coordinates (no road snapping)
            target_location = carla.Location(x=world_x, y=world_y, z=world_z)
            target_waypoint = FreeWaypoint(target_location, vehicle_transform.rotation)
            
            # Get LiDAR densities for logging (use correct key names from cameras dict)
            cam_names = list(cameras.keys())
            d_front = current_projection_counts.get('front', current_projection_counts.get('camera_front', 0))
            d_left = current_projection_counts.get('left', current_projection_counts.get('camera_left', 0))
            d_right = current_projection_counts.get('right', current_projection_counts.get('camera_right', 0))
            
            # Step 4: Use PID controller for smooth control
            control = pid_controller.run_step(config.target_speed_kmh, target_waypoint)
            vehicle.apply_control(control)
            
            # Log with world coordinates and LiDAR densities
            print(f"[Tick {tick_count}] {direction} | Pos: ({vehicle_loc.x:.1f}, {vehicle_loc.y:.1f}, {vehicle_loc.z:.1f}) | Target: ({world_x:.1f}, {world_y:.1f}) | LiDAR [F:{d_front:.0f} L:{d_left:.0f} R:{d_right:.0f}] | Steer: {control.steer:+.3f} | Throttle: {control.throttle:.2f}")
        
        update_spectator_view(world, vehicle)

except KeyboardInterrupt:
    print("\n✗ Interrupted")

print(f"\n✓ Done! {frame_counter} frames saved")

✓ Running 30.0s | Target speed: 20.0 km/h | Controller: PID

[Tick 1] F | Pos: (35.7, 24.8, 0.3) | Target: (45.7, 24.8) | LiDAR [F:6544 L:7538 R:7819] | Steer: +0.000 | Throttle: 0.75
[Tick 2] F | Pos: (35.7, 24.8, 0.1) | Target: (45.7, 24.8) | LiDAR [F:6355 L:7657 R:7735] | Steer: +0.000 | Throttle: 0.75
[Tick 3] F | Pos: (35.7, 24.8, 0.0) | Target: (45.7, 24.8) | LiDAR [F:6447 L:7562 R:7609] | Steer: +0.000 | Throttle: 0.75
[Tick 4] F | Pos: (35.7, 24.8, -0.1) | Target: (45.7, 24.8) | LiDAR [F:6364 L:7370 R:7332] | Steer: +0.000 | Throttle: 0.75
[Tick 5] F | Pos: (35.7, 24.8, -0.1) | Target: (45.7, 24.8) | LiDAR [F:6333 L:7568 R:7746] | Steer: +0.000 | Throttle: 0.75
[Tick 6] F | Pos: (35.7, 24.8, -0.1) | Target: (45.7, 24.8) | LiDAR [F:6304 L:7618 R:7417] | Steer: +0.000 | Throttle: 0.75
[Tick 7] F | Pos: (35.7, 24.8, -0.1) | Target: (45.7, 24.8) | LiDAR [F:6414 L:7435 R:7860] | Steer: +0.000 | Throttle: 0.75
[Tick 8] F | Pos: (35.7, 24.8, -0.0) | Target: (45.7, 24.8) | LiDAR [F:646

---

## 3. Cleanup & Shutdown

Stop sensors, destroy actors, restore async mode.

In [4]:
restore_async_mode(world, original_settings, traffic_manager)

for cam in cameras.values():
    cam.stop()
lidar.stop()
time.sleep(0.5)

for cam in cameras.values():
    cam.destroy()
lidar.destroy()
vehicle.destroy()
world.tick()

print(f"✓ Cleanup done | Output: {output_dir}/lidar_projection/")

✓ Cleanup done | Output: output/2026-02-02_22-45-27/lidar_projection/
